In [1]:
import os
import pickle
import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
from sklearn import svm
import matplotlib.pyplot as plt
from Haldane_anis_supervise_class import *
from partial import *

In [2]:
bond = 50  

In [3]:
L = 3 # <------------ specify the length of the chain each time (change this from 1 to 10)
keep = [2,3,4] # <------------ specify the particles to keep each time (based on your keeping sites)
H = Haldan_anis(L = L, bond = bond, keep = keep)

In [4]:
test_kernel_scalable = H.generate_test_kernel()

Gram Test Scalable: 100%|██████████| 1600/1600 [00:03<00:00, 515.85it/s]

Kernel test set saved to ./dataset_L=51_bond=50_partial(keeping 3 sites)/kernel_test_scalable.hdf5
